# This is Jeopardy!

In [1]:
import pandas as pd
import string
import re
import random
pd.set_option('display.max_colwidth', None)

In [2]:
#Load the data into a DataFrame and investigate its contents.
df = pd.read_csv("jeopardy.csv")

#inspect the dataframe attributes
firstrow_list = df.columns.tolist()

#print(firstrow_list)

#Rename the columns 
df.columns = ['Show_Number', 'Air_Date', 'Round', 'Category', 'Value', 'Question', 'Answer']

#print(df.head())

#print(df.Question.head())

testList = ["King", "England"]


In [3]:
# Write a function that filters the dataset for questions that contains all of the words in a list of words.
def filter_question( dataframe, words_list):
    """This function filters the dataset for questions that contains all of the words in a list of words.
    Args:
        data, words: List of words in question coulumn
    Returns:
        [type]: Dataframe 
    """
    filter = lambda x: all(word.lower() in x.lower() for word in words_list)
    return dataframe.loc[dataframe["Question" ].apply(filter)]

print(filter_question(df,testList))

        Show_Number    Air_Date             Round               Category  \
4953           3003  1997-09-24  Double Jeopardy!           "PH"UN WORDS   
6337           3517  1999-12-14  Double Jeopardy!                    Y1K   
9191           3907  2001-09-04  Double Jeopardy!         WON THE BATTLE   
11710          2903  1997-03-26  Double Jeopardy!       BRITISH MONARCHS   
13454          4726  2005-03-07         Jeopardy!  A NUMBER FROM 1 TO 10   
...             ...         ...               ...                    ...   
208295         4621  2004-10-11         Jeopardy!            THE VIKINGS   
208742         4863  2005-11-02  Double Jeopardy!         BEFORE & AFTER   
213870         5856  2010-02-15  Double Jeopardy!                 URANUS   
216021         1881  1992-11-09  Double Jeopardy!         HISTORIC NAMES   
216789         5070  2006-09-29  Double Jeopardy!        ANCIENT HISTORY   

         Value  \
4953      $200   
6337      $800   
9191      $800   
11710     $600 

## Clean the dataset
Clean the new_value column values before converting them to float. 

In [4]:
#create new column with without the dollar sign
df['new_Value'] = df.Value.str.strip('$')
#print(df.head(25))

In [5]:
# Inspecting the string values in the dataset
# In addition to string with 'None' value, there are values with coma to clean before converting to type float
distinct_dtr_values = []
for i in range(len(df['new_Value'])):
    if (len(df['new_Value'].iloc[i]) > 4) & (df['new_Value'].iloc[i] not in distinct_dtr_values):
        distinct_dtr_values.append(df['new_Value'].iloc[i])

print(distinct_dtr_values)
        

    

['2,000', '3,200', '5,000', '1,000', '1,500', '1,200', '4,800', '1,800', '1,100', '2,200', '3,400', '3,000', '4,000', '1,600', '6,800', '1,900', '3,100', '1,400', '2,800', '8,000', '6,000', '2,400', '12,000', '3,800', '2,500', '6,200', '10,000', '7,000', '1,492', '7,400', '1,300', '7,200', '2,600', '3,300', '5,400', '4,500', '2,100', '3,600', '2,127', '4,400', '3,500', '2,900', '3,900', '4,100', '4,600', '10,800', '2,300', '5,600', '1,111', '8,200', '5,800', '7,500', '1,700', '9,000', '6,100', '1,020', '4,700', '2,021', '5,200', '3,389', '4,200', '2,001', '1,263', '4,637', '3,201', '6,600', '3,700', '2,990', '5,500', '14,000', '2,700', '6,400', '8,600', '6,300', '3,989', '8,917', '9,500', '1,246', '6,435', '8,800', '2,222', '2,746', '10,400', '7,600', '6,700', '5,100', '13,200', '4,300', '1,407', '12,400', '5,401', '7,800', '1,183', '1,203', '13,000', '11,600', '14,200', '1,809', '8,400', '8,700', '11,000', '5,201', '1,801', '3,499', '5,700', '4,008', '2,344', '2,811', '18,000', '1,777

In [6]:
# Clean the dataset by removing the comma in the string values with punctuation and by replacing the None value by 0.
df['new_Value'] = df['new_Value'].apply(lambda v: v.translate(v.maketrans('','', string.punctuation)) if v != 'None' else 0 )



## Convert the string values to float

In [7]:
# Convert the new_Value column values to floats
df['new_Value'] = df['new_Value'].astype(float)

print(df['new_Value'])

0          200.0
1          200.0
2          200.0
3          200.0
4          200.0
           ...  
216925    2000.0
216926    2000.0
216927    2000.0
216928    2000.0
216929       0.0
Name: new_Value, Length: 216930, dtype: float64


In [8]:
#average value of the new column
def average_value(col):
    """This function calculate the average value of a col column which values are converted to floats 

    Args:
        col : a column with float values

    Returns:
        [type]: float
    """
    avg = round(col.mean(), 2)
    return avg

print(average_value(df.new_Value))

739.99


In [9]:
#Write a function that returns the count of the unique answers to all of the questions in a dataset.


def get_count_unique(data, word, col_label1, col_label2):
    """This function filters the dataset for all the questions that contain the given word and returns the count of the unique answers to all of the questions in the dataset.

    Args:
        dataset, word, col_label1, col_label2: a word in the question column

    Returns:
        [type]: Dataframe 
    """
    indice_list = []
    answer_list = []
    question_list = []
   

    # get the indexes of the questions that contain the given word    
    for i in data.index:
        if data.loc[i, col_label1].find(word) == True:
            indice_list.append(i)
    #get the correspondent Answer for each selected indice
    for j in indice_list:
        question_list.append(data.loc[j, col_label1])
        answer_list.append(data.loc[j, col_label2])

    # return  a Series containing counts of unique rows in the DataFrame of Answer_list
    unique_answer = pd.DataFrame(answer_list, question_list).reset_index()
    
    # rename the columns
    unique_answer.columns = ['question', 'answer']
    
    #count unique values
    val_counts = unique_answer.value_counts()
    
    #converting to DataFrame and assigning the column names.
    df_val_counts = pd.DataFrame(val_counts).reset_index()
    df_val_counts.columns = ['question', 'answer', 'unique_value']
                                      
    return df_val_counts


            
print(get_count_unique(df, 'King', 'Question', 'Answer'))

                                                                                                                   question  \
0   "Kings play chess on finely grained sand" is a mnemonic device used to help remember a system devised by this scientist   
1                                                                "Kingdom" that Khrushchev couldn't visit in 1959 U.S. trip   
2                                                                     "King" of the dinosaurs who starred in "My Fair Lady"   
3                                                                                              "King" of nonsense poets (4)   
4                   "King of the World" chronicles "The Rise of an American Hero", this legendary boxer & minister of Islam   
5                    "King of the Waltz" Johann Strauss Jr. wrote in other dance forms too, like the "Tritsch-Tratsch" this   
6                         "King of the Hill"'s guitar-playing comedian Celery Head is a spoof of this red-haire

## Dataset Exploration

First, we are going to investigate the ways in which questions change over time by filtering by the date. Then we are going to count how many questions from the 90s use the word ```"Computer"``` compared to questions from the 2000s.

Next, we are going to check if there is a connection between the round and the category. Are you more likely to find certain categories, like ```"Literature"``` in Single Jeopardy or Double Jeopardy?

In [10]:
print(get_count_unique(df, 'Computer', 'Question', 'Air_Date' ))

                                                                                                     question  \
0  "Computer, Communication, Compatibility" were combined to describe the business of this Silicon Valley co.   

       answer  unique_value  
0  2008-02-25             1  


In [11]:
# this version uses a regex expression to only match whole words, not substrings
def filter_questions_regex(string_list):
    filter = lambda question : all(re.search(r'\b' + string.lower() + r'\b', question.lower()) for string in string_list)
    return df[df.Question.apply(filter)]

print(filter_questions_regex(testList))

        Show_Number    Air_Date             Round  \
4953           3003  1997-09-24  Double Jeopardy!   
6337           3517  1999-12-14  Double Jeopardy!   
9191           3907  2001-09-04  Double Jeopardy!   
11710          2903  1997-03-26  Double Jeopardy!   
13454          4726  2005-03-07         Jeopardy!   
...             ...         ...               ...   
201168         3515  1999-12-10         Jeopardy!   
204778         5899  2010-04-15  Double Jeopardy!   
208742         4863  2005-11-02  Double Jeopardy!   
213870         5856  2010-02-15  Double Jeopardy!   
216021         1881  1992-11-09  Double Jeopardy!   

                             Category   Value  \
4953                     "PH"UN WORDS    $200   
6337                              Y1K    $800   
9191                   WON THE BATTLE    $800   
11710                BRITISH MONARCHS    $600   
13454           A NUMBER FROM 1 TO 10   $1000   
...                               ...     ...   
201168              

In [12]:
# get most common answer for questions selected by words from a list
def get_most_frequente_answer(words_list):
    filtered = filter_question(df, words_list)
    answers = filtered.groupby('Answer').Show_Number.count().reset_index()
    most_selected = answers[answers.Show_Number == answers.Show_Number.max()]
    return most_selected

#print(get_most_frequente_answer(['King']))
print(get_most_frequente_answer(['Computer']))
#print(get_most_frequente_answer(testList))

   Answer  Show_Number
17  Apple           12
84    IBM           12


### Investigate the ways in which questions change over time by filtering by the date

In [13]:
#define decades
def decades(mydate):
    mydate =pd.to_numeric(mydate)
    decade70s =  (1970 <= mydate < 1980 and '1970s') 
    decade80s =  (1980 <= mydate < 1990 and '1980s') 
    decade90s =  (1990 <= mydate < 2000 and '1990s')
    decade00s =  (2000 <= mydate < 2010 and '2000s')
    decade10s =  (2010 <= mydate < 2020 and '2010s')
    return decade70s  or decade80s  or decade90s  or decade00s  or decade10s

# Number of questions from the 90s use the word "Computer" compared to questions from the 2000s
def filter_by_decade(words_list):
    """This function filters the dataset for all the questions that contain the givena list of strings and returns 
    the count of the questions by decade.

    Args:
       a list of strings

    Returns:
        [type]: Dataframe 
    """
    
    filtered = filter_question(df, words_list)
    
    #extract years in new column as numeric values
    filtered['Air_Date'] = filtered['Air_Date'].str.replace('-', '')
    filtered['year'] = pd.to_numeric(pd.DatetimeIndex(filtered['Air_Date']).year)
    
    # create a decades columns in the the filtered DataFrame
    filtered['decades'] = filtered.year.apply( lambda year: decades(year)  )
    
    # define decades DataFrame
    decadeDataFrame = filtered.groupby('decades').Question.count().reset_index()
    
    return decadeDataFrame

print(filter_by_decade(['Computer']))

  decades  Question
0   1980s         6
1   1990s        98
2   2000s       268
3   2010s        59


<ipython-input-13-8c1b984d1bfe>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['Air_Date'] = filtered['Air_Date'].str.replace('-', '')
<ipython-input-13-8c1b984d1bfe>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['year'] = pd.to_numeric(pd.DatetimeIndex(filtered['Air_Date']).year)
<ipython-input-13-8c1b984d1bfe>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

### Seaching a connection between the round and the category

In [14]:
# Write a function that filters the dataset for questions that contains all of the words in a list of words.
def filter_category( dataframe, words_list):
    """This function filters the dataset for questions that contains all of the words in a list of words.
    Args:
        data, words: List of words in question coulumn
    Returns:
        [type]: Dataframe 
    """
    filter = lambda x: all(word.lower() in x.lower() for word in words_list)
    return dataframe.loc[dataframe["Category" ].apply(filter)]

#print(filter_category(df,["Literature"]))

def filter_by_round(words_list):
    """This function filters the dataset for all the categories that contain the given list of strings and returns 
    the count of the categories by round.

    Args:
       a list of strings

    Returns:
        [type]: Dataframe 
    """
    
    filtered = filter_category(df, words_list)
    
    
    # define decades DataFrame
    grouped_df = filtered.groupby('Round').Category.count().reset_index()
    
    return grouped_df

#print(filter_by_round(["Literature"]))

In [15]:
def get_gl_knowledge_statements():
    
    #build a list of question-answer
    statements = []
    question_list = []
    answer_list = []
    for i in df[:10].index:
        question_list.append(df.loc[i, 'Question']) 
        answer_list.append(df.loc[i, 'Answer'])

    statements = zip(question_list, answer_list)
    statements = list(zip(question_list, answer_list))
    
    return  statements

#print(get_gl_knowledge_statements())

def get_literature_statements():
    
    litf = filter_category(df,["Literature"])
    
    #build a list of ten question-answer
    statements = []
    question_list = []
    answer_list = []
    for i in litf[:10].index:
        question_list.append(litf.loc[i, 'Question'])
        answer_list.append(litf.loc[i, 'Answer'])

    statements = zip(question_list, answer_list)
    statements = list(zip(question_list, answer_list))
    
    return  statements

print(get_literature_statements())
    

[('In one Grimm tale, 12 princesses dance these to pieces in an underground castle', 'their shoes'), ('Anne Shirley leaves the orphanage to live on this "colorful" farm in Avonlea', 'Green Gables'), ('"The Comic Adventures of" this elderly woman "and her Dog" were first published in 1805', 'Old Mother Hubbard'), ('Dinarzade is the younger sister of this woman known for her nocturnal stories', 'Shahrazad'), ('Madeline is one of "twelve little girls in two straight lines" who attend a school in this city', 'Paris'), ('The poem called the "Garibnameh" contains 11,000 masnavi, which we know as these rhymed 2-line units of verse', 'couplets'), ('A religious kaside poem praised God, this man or his son-in-law, Ali Ibn Abi Talib, the fourth caliph', 'Muhammad'), ('Sultan Abdulhamid II\'s censorship hindered Ottoman writers until this "youthful" group\'s 1908 revolution', 'the Young Turks'), ('Peter Ustinov directed & starred in the film version of the Yasar Kemal novel "Memed, My" this predat

In [16]:
def play_literature_quiz():
    
    # Get literature statements
    literature_statements = get_literature_statements()
    
    # Randomise literature statements
    random.shuffle(literature_statements)
    
    # set player score to 0
    score = 0
    
    # Show tof statements using a loop
    for s in literature_statements:
        
        # present statement
        print("Literature: " + s[0])
        
        # user enter guess
        guess = input("Enter your answer: ")
        # check if guess is correct
        if guess == s[1]:
            print("Correct!")
            # update score
            score += 1
        else:
            print("Incorrect :(")
            
    # Show final score
    print("Your final score is: " + str(score))


def play_gl_knowledge_quiz():
    
    # Get general knowledge statements
    gl_knowledge_statements = get_gl_knowledge_statements()
    
    # Randomise general knowledge statements
    random.shuffle(gl_knowledge_statements)
    
    # set player score to 0
    score = 0
    
    # Show tof statements using a loop
    for s in gl_knowledge_statements:
        
        # present statement
        print("General Knowledge: " + s[0])
        
        # user enter guess
        guess = input("Enter your answer: ")
        # check if guess is correct
        if guess == s[1]:
            print("Correct!")
            # update score
            score += 1
        else:
            print("Incorrect :(")
            
    # Show final score
    print("Your final score is: " + str(score))

In [17]:
def main_menu():
    
    print("+--------------------------------+")
    print("|    Welcome to QUIZ JEOPARDY!   |")
    print("+--------------------------------+")
    print("| Please select an option:       |")
    print("|                                |")
    print("| 1. Play Literature quiz        |")
    print("| 2. Play General Knowledge quiz |")
    print("| 0. Quit                        |")
    print("+--------------------------------+")
    choice = input("Enter 1, 2, or 0: ")
    if choice == "1":
        play_literature_quiz()
    elif choice == "2":
        play_gl_knowledge_quiz()
    else:
        print("Thank for your laying!")
        quit()
    return choice

main_menu()
    

+--------------------------------+
|    Welcome to QUIZ JEOPARDY!   |
+--------------------------------+
| Please select an option:       |
|                                |
| 1. Play Literature quiz        |
| 2. Play General Knowledge quiz |
| 0. Quit                        |
+--------------------------------+
Enter 1, 2, or 0: 0
Thank for your laying!


'0'